In [1]:
%matplotlib inline
import sys
sys.path.append('../')
from src.LRGSG_package.LRGSG import *

In [7]:
ssize_list = [2**k for k in range(4, 8)]
nrep = [range(2**(16-2*k)) for k in range(4, 8)]
pval_list = [0.01, 0.09, 0.098, 0.100, 0.102, 0.104, 0.24]
print('list of sizes', ssize_list, '\nlist of flip probs', pval_list, '\nlist of replicas', nrep)

list of sizes [16, 32, 64, 128] 
list of flip probs [0.01, 0.09, 0.098, 0.1, 0.102, 0.104, 0.24] 
list of replicas [range(0, 256), range(0, 64), range(0, 16), range(0, 4)]


In [8]:
d_lmin = []

for i,L in enumerate(ssize_list[:1]):
    nedges = 2*L**2
    for p in pval_list:
        lmin = []
        for nr in tqdm(nrep[i], desc=f"replicas for L={L}, p={p}"):
            G = nx.grid_2d_graph(L, L, periodic=True)
            ransample = random.sample(range(nedges), int(p*nedges))
            #
            all_weights = {e: 1 for e in G.edges()}
            neg_weights = {e: -1 for i,e in enumerate(G.edges()) if i in ransample}
            #
            nx.set_edge_attributes(G, values=all_weights, name='weight')
            nx.set_edge_attributes(G, values=neg_weights, name='weight')
            slapl = get_graph_lapl(G)
            try:
                lmin_tmp, _ = eigsh(slapl, k=1, which='SM', tol=1e-10)
            except ArpackNoConvergence:
                pass
            lmin.append(lmin_tmp)
        d_lmin.append({'L': L, 'p':p, 'lmin': np.array(lmin).flatten()})
        np.savetxt(f"{da}")

replicas for L=16, p=0.1:  39%|███▉      | 100/256 [00:27<00:43,  3.61it/s]


KeyboardInterrupt: 

In [9]:
d_lmin

[{'L': 16,
  'p': 0.01,
  'lmin': [array([2.32441922e-16]),
   array([-1.41653587e-16]),
   array([2.06375322e-16]),
   array([4.39877457e-16]),
   array([8.39124499e-17]),
   array([7.176969e-16]),
   array([-2.75800116e-16]),
   array([1.32950122e-15]),
   array([1.89033019e-15]),
   array([2.14053233e-16]),
   array([-5.94311244e-16]),
   array([-5.30082504e-16]),
   array([-6.25694838e-16]),
   array([3.99763654e-16]),
   array([1.30376989e-16]),
   array([-2.02272229e-16]),
   array([2.74413828e-16]),
   array([1.34598854e-16]),
   array([-1.1460538e-16]),
   array([-3.8319512e-16]),
   array([4.9479386e-16]),
   array([-5.62482006e-17]),
   array([-4.50620207e-16]),
   array([-2.85916913e-16]),
   array([-1.05168098e-17]),
   array([2.76891078e-16]),
   array([-4.34308582e-16]),
   array([-1.98665995e-16]),
   array([-1.58146871e-16]),
   array([-8.61983928e-17]),
   array([-5.08966708e-16]),
   array([-1.93621753e-16]),
   array([-1.5837986e-16]),
   array([4.50038406e-16]),
   

In [ ]:
L = 16
nrep = [range(2**(17-2*k)) for k in range(4, 8)]
p = 0.1
nedges = 2*L**2
lmin = []
for nr in tqdm(nrep[0]):
    G = nx.grid_2d_graph(L, L, periodic=True)
    ransample = random.sample(range(nedges), int(p*nedges))
    #
    all_weights = {e: 1 for e in G.edges()}
    neg_weights = {e: -1 for i,e in enumerate(G.edges()) if i in ransample}
    #
    nx.set_edge_attributes(G, values=all_weights, name='weight')
    nx.set_edge_attributes(G, values=neg_weights, name='weight')
    slapl = slaplacian_matrix(G).asfptype()
    try:
        lmin_tmp, _ = eigsh(slapl, k=1, which='SM', tol=1e-10)
    except ArpackNoConvergence:
        pass
    lmin.append(lmin_tmp)

In [ ]:
plt.hist(np.array(lmin).flatten(), bins=20)
plt.yscale('log')

In [ ]:
colors = ['r' if G[u][v]['weight'] == -1 else 'k' for u, v in G.edges()]
plt.figure(figsize=(10, 10))
pos = {(x,y):(y,-x) for x,y in G.nodes()}
nx.draw(G, edge_color=colors, pos=pos, node_size=10, width=5)

# JUNK

In [ ]:
L = 4
G = nx.grid_2d_graph(L, L, periodic=True)
L1 = slaplacian_matrix(G)
L2 = nx.laplacian_matrix(G)
L1.asfptype() - L2.asfptype()

In [ ]:
ransample = random.sample(range(2*L*L), 2*L*L)
neg_weights = {e: -1 for i,e in enumerate(G.edges()) if i in ransample}
nx.set_edge_attributes(G, values=neg_weights, name='weight')
L1 = slaplacian_matrix(G)
L2 = nx.laplacian_matrix(G)
L1.asfptype() - L2.asfptype()